<a href="https://colab.research.google.com/github/missriri/IS-II/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam
#from keras import np_utils
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg






tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="QvDpQHBoHShaC0LOwPbc")
project = rf.workspace("nir").project("recycling-codes-detection-v2")
dataset = project.version(2).download("tfrecord")


In [3]:
# Define a parsing function to extract data from the TFRecords
@tf.function
def parse_tfrecord_fn(example):
    feature_description = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/format': tf.io.FixedLenFeature([], tf.string),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
        # Add more feature descriptions as needed
    }

    example = tf.io.parse_single_example(example, feature_description)

    # Decode and preprocess the image
    image = tf.image.decode_jpeg(example['image/encoded'], channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)

    # Extract the label IDs
    label_ids = example['image/object/class/label']

    # Define your label map based on the class names and IDs
    label_map = {
        1: "1_PET",
        2: "2_PEHD",
        3: "3_PVC",
        4: "4_PELD",
        5: "5_PP",
        6: "6_PS",
        7: "7_Other"
    }

    # Create a StaticHashTable for label conversion
    label_table = tf.lookup.StaticHashTable(
        tf.lookup.KeyValueTensorInitializer(
            list(label_map.keys()), list(label_map.values()),
            key_dtype=tf.int64, value_dtype=tf.string),
        default_value="Unknown"
    )

    # Convert label IDs to class names using the label map
    class_names = label_table.lookup(label_ids.values)

    # Print the class names to verify the conversion
    print("Class Names:", class_names)

    return image, class_names




In [ ]:
tfrecord_files = 'recycling-codes-detection-v2-2/train/*.tfrecord'

# Create a TFRecord dataset
dataset = tf.data.TFRecordDataset(tf.data.Dataset.list_files(tfrecord_files))

# Apply the parsing function to the dataset
parsed_dataset = dataset.map(parse_tfrecord_fn)

# You can now iterate through the dataset and access the parsed data
for image, class_names in parsed_dataset:
    # Access the image and class names for each example
    print("Image Shape:", image.shape)
    print("Class Names:", class_names)

In [ ]:
# Iterate through the dataset and display the first image
for image, class_names in parsed_dataset:
    # Display the first image
    plt.imshow(image.numpy())  # Convert the TensorFlow tensor to a NumPy array

    # Convert byte strings to regular strings and join them
    class_names_str = ', '.join([name.decode() for name in class_names.numpy()])

    plt.title("Class Names: " + class_names_str)  # Use the decoded class names
    plt.show()
    break  # Display only the first image

In [ ]:
#dataset 2

from roboflow import Roboflow
rf = Roboflow(api_key="QvDpQHBoHShaC0LOwPbc")
project = rf.workspace("thesis-dataset-5dsvz").project("ric")
dataset2 = project.version(1).download("tfrecord")



In [7]:
print (dataset2)
print (dataset)

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>


In [ ]:
import os
import shutil
import tensorflow as tf
import re
import uuid


def convert_label_ids(label_ids):
    def convert(label_id):
        if not label_id.indices.shape[0] == 0:
            class_id = label_id.values[0].numpy()
            class_name = label_map1[class_id]
        else:
            class_name = ""  # Handle empty label_id
        if class_name in class_mapping:
            class_name = class_mapping[class_name]
        return class_name

    label_names = tf.py_function(convert, inp=[label_ids], Tout=tf.string, name="convert_label_ids")
    return label_names

def strip_class_name(class_name):
  class_name_as_string = tf.strings.as_string(class_name)
  class_name_string = tf.strings.regex_replace(class_name_as_string, r"[^\d-]", "")  # Apply regex substitution
  return  class_name_string


def parse_tfrecord_fn(example, label_map):
    feature_description = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/format': tf.io.FixedLenFeature([], tf.string),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
        # Add more feature descriptions as needed
    }

    example = tf.io.parse_single_example(example, feature_description)

    # Decode and preprocess the image
    image = tf.image.decode_jpeg(example['image/encoded'], channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)

    # Extract the label IDs
    label_ids = example['image/object/class/label']
    print(label_ids)

    # Use tf.py_function to convert label IDs to class names
    class_names = tf.py_function(convert_label_ids, inp=[label_ids], Tout=tf.string, name="convert_label_ids")
    print("----class name----\n",class_names)

    return image, class_names

# Define the paths to the TFRecord files for dataset 1 and dataset 2
tfrecord_files1 = 'recycling-codes-detection-v2-2/train/*.tfrecord'
tfrecord_files2 = 'RIC-1/train/*.tfrecord'

# Create a TFRecord dataset for dataset 1
dataset1 = tf.data.TFRecordDataset(tf.data.Dataset.list_files(tfrecord_files1))

# Create a TFRecord dataset for dataset 2
dataset2 = tf.data.TFRecordDataset(tf.data.Dataset.list_files(tfrecord_files2))

# Define a mapping between class labels in dataset 1 and dataset 2
class_mapping = {
    "2_PEHD": "2_PEHD",  # No change for dataset 1 labels
    "4_PELD": "4_PELD",  # No change for dataset 1 labels
    "2_HDPE": "2_PEHD",  # Merge dataset 2 "2_HDPE" into dataset 1 "2_PEHD"
    "4_LDPE": "4_PELD",  # Merge dataset 2 "4_LDPE" into dataset 1 "4_PELD"
}

# Define the label maps for both datasets
label_map1 = {
    1: "1_PET",
    2: "2_PEHD",
    3: "3_PVC",
    4: "4_PELD",
    5: "5_PP",
    6: "6_PS",
    7: "7_Other"
}

label_map2 = {
    1: "1_PET",
    2: "2_HDPE",
    3: "4_LDPE",
    4: "5_PP",
}

# Parse the datasets using the parse_tfrecord_fn and label maps
parsed_dataset1 = dataset1.map(lambda example: parse_tfrecord_fn(example, label_map1))
parsed_dataset2 = dataset2.map(lambda example: parse_tfrecord_fn(example, label_map2))

# Define the directory where the combined dataset will be saved
combined_dataset_dir = "combined-dataset"

# Create subfolders for each unique class label (taking into account class mapping)
unique_labels = set()

# Parse dataset1 and add unique labels to the set
for example in dataset1:
    image, class_names = parse_tfrecord_fn(example, label_map1)
    if isinstance(class_names, int):
        class_name = str(class_names)
    elif isinstance(class_names, bytes):
        class_name = class_names.decode()
    elif isinstance(class_names, str):
        class_name = class_names
    else:
        class_name = ''

    if class_name in class_mapping:
        class_name = class_mapping[class_name]
    if class_name:
        unique_labels.add(class_name)

# Parse dataset2 and add unique labels to the set
for example in dataset2:
    image, class_names = parse_tfrecord_fn(example, label_map2)
    if isinstance(class_names, int):
        class_name = str(class_names)
    elif isinstance(class_names, bytes):
        class_name = class_names.decode()
    elif isinstance(class_names, str):
        class_name = class_names
    else:
        class_name = ''

    if class_name:
        unique_labels.add(class_name)

# Create subfolders for each unique class label
for label in unique_labels:
    label_dir = os.path.join(combined_dataset_dir, label)
    os.makedirs(label_dir, exist_ok=True)

# Create a function to extract the class name from a label
def extract_class_name(label):
    print('unsplit class name is',label.numpy().decode())
    class_name = label.numpy().decode().split('_')[-1]
    print('class name is:',class_name)
    return class_name

# Parse and copy images to their respective class label subfolders
for example in dataset1:
    image, label = parse_tfrecord_fn(example, label_map1)
    class_name = extract_class_name(label)

    unique_labels.add(class_name)

    # Debugging information
    print('---dataset 1---')
    print(f"Image class: {class_name}")
    print(f"Image shape: {image.shape}")
    print(f"Label: {label.numpy().decode()}")


    if class_name in class_mapping:
        class_name = class_mapping[class_name]

    # save the image to the corresponding subfolder
    image_bytes = image.numpy().tobytes()
    image_hash = hash(image_bytes)
    image_filename = f"{class_name}_{image_hash}.jpg"

    class_dir = os.path.join(combined_dataset_dir, class_name)
    os.makedirs(class_dir, exist_ok=True) # Create the directory if it doesn't exist
    output_path = os.path.join(class_dir, image_filename)

    # Convert the image data type to uint8
    image = tf.image.convert_image_dtype(image, tf.uint8)
    # Encode the image as JPEG
    image_data = tf.image.encode_jpeg(image)

    with open(output_path, "wb") as img_file:
        img_file.write(image_data.numpy())




for example in dataset2:
    image, class_names = parse_tfrecord_fn(example, label_map2)
    class_name = extract_class_name(label)

    if class_name in class_mapping:
        class_name = class_mapping[class_name]

     # Debugging information
    print('---dataset 1---')
    print(f"Image class: {class_name}")
    print(f"Image shape: {image.shape}")
    print(f"Label: {label.numpy().decode()}")

    image_bytes = image.numpy().tobytes()
    image_hash = hash(image_bytes)
    image_filename = f"{class_name}_{image_hash}.jpg"

    class_dir = os.path.join(combined_dataset_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)
    output_path = os.path.join(class_dir, image_filename)

    # Convert the image data type to uint8
    image = tf.image.convert_image_dtype(image, tf.uint8)
    # Encode the image as JPEG
    image_data = tf.image.encode_jpeg(image)

    with open(output_path, "wb") as img_file:
        img_file.write(image_data.numpy())



In [10]:
# Raw images dataset
from google.colab import drive
drive.mount('/content/drive')

# Change directory to where images are stored in  Google Drive
import os
os.chdir('/content/drive/My Drive/plastics')

!ls



Mounted at /content/drive
HDPEM  PET


In [30]:
import glob
import glob
# combine all datasets together then split them into training, testing and validation
# datasets

combined_dataset_dir = "combined-dataset"

#get a list of image files
image_files = glob.glob(os.path.join(combined_dataset_dir, "*", "*.jpg"))
#extract labels
labels = [os.path.basename(os.path.dirname(file_path)) for file_path in image_files]





In [37]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


#initialize IDG for augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [49]:
# Create generators for loading images from directories and performing data augmentation
train_dir = os.path.join(combined_dataset_dir, "train")
test_dir = os.path.join(combined_dataset_dir, "test")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)





In [50]:
# Define batch size and image size
batch_size = 32
image_size = (150, 150)

# Flow training images in batches using train_datagen
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(640, 640),  # Update the target size
    batch_size=batch_size,
    class_mode='categorical'
)

# Flow validation images in batches using validation_datagen
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(640, 640),  # Update the target size
    batch_size=batch_size,
    class_mode='categorical'
)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [51]:
train_dir, test_dir = tf.keras.utils.image_dataset_from_directory(
    combined_dataset_dir,
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 0 files belonging to 2 classes.
Using 0 files for training.
Using 0 files for validation.


ValueError: ignored